In [1]:
#!pip install geopandas
#!pip install geopy
#!pip install folium
#!pip install openpyxl

In [ ]:
# notes
# changed the heading -- some of the inconsistencies that are more human readable, are less machine readable

In [1]:
import geopandas
import geopy
import folium

import os
import re

import pandas as pd
import seaborn as sns

from datetime import date

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# change color of circle based on: occupation, person name, year, decade, etc.

In [2]:
#df = pd.read_excel('/home/stephbuon/projects/THADS/ai_life_members.xlsx', sheet_name=None)
df = pd.read_csv('/home/stephbuon/projects/THADS/ai_life_members_1882.csv')

df = df.head(10)

In [3]:
df['decade'] = df['YEAR'].apply(lambda x: x - (x % 10))
df['address'] = df['STREET'] + ',' + df['CITY'] + ',' + df['STATE'] + ',' + 'United States'
df = df[['address', 'decade']]

In [4]:
today = date.today()
d_today = today.strftime("%m-%d-%Y")

In [5]:
def import_data():
    pass
# data_folder = os.path.join("source_data", "text_files")


def address_standardization(row):
    row = str(row)

    if re.search(r'(?<=\d 2).*(?=d )', row):
        return re.sub(r'(?<=\d 2).*(?=d )', 'n', row)

    elif re.search(r'(?<=\d [0-9]2).*(?=d )', row):
        return re.sub(r'(?<=\d [0-9]2).*(?=d )', 'n', row)

    elif re.search(r'(?<=\d [0-9][0-9]2).*(?=d )', row):
        return re.sub(r'(?<=\d [0-9][0-9]2).*(?=d )', 'n', row)

    elif re.search(r'(?<=\d[0-9][0-9]2).*(?=d )', row): # I don't think this worked
        return re.sub(r'(?<=\d[0-9][0-9]2).*(?=d )', 'n', row)

    elif re.search(r'(?<=\d 3).*(?=d )', row):
        return re.sub(r'(?<=\d 3).*(?=d )', 'r', row)

    elif re.search(r'(?<=\d [0-9]3).*(?=d )', row):
        return re.sub(r'(?<=\d [0-9]3).*(?=d )', 'r', row)

    elif re.search(r'(?<=\d [0-9][0-9]3).*(?=d )', row):
        return re.sub(r'(?<=\d [0-9][0-9]3).*(?=d )', 'r', row)

    else:
        return row


def export_unmapped_data(df, export_dir, d_today):

    missing_data = df.isnull().any(axis=1)
    unmapped_addresses = df[missing_data]
    
    if not os.path.exists(export_dir):
        os.mkdir(export_dir)

    exp = os.path.join(export_dir, 'unmapped_addresses_' + d_today + ".csv")

    unmapped_addresses.to_csv(exp)


def geolocate(df, export_dir, d_today, clean_data):

    if clean_data == True:
        df['address'] = df['address'].apply(address_standardization)

    locator = Nominatim(user_agent = "myGeocoder")

    # adding a 1 second delay between requesets to avoid spamming the API
    geocode = RateLimiter(locator.geocode, min_delay_seconds = 1)

    df['location'] = df['address'].apply(geocode)
    df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

    export_unmapped_data(df, export_dir, d_today)

    # code will throw an error if you try and explode or map an empty value
    df = df.dropna()

    df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index = df.index)

    exp = os.path.join(export_dir, "geolocated_data_" + d_today + ".csv")

    df.to_csv(exp)
    
    return(df)


In [6]:
df = geolocate(df, '/home/stephbuon/projects/THADS/THADS_map', d_today, clean_data = True)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('East 123d street,New York,NY,United States',), **{}).
Traceback (most recent call last):
  File "/home/stephbuon/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/stephbuon/miniconda/lib/python3.8/site-packages/urllib3/connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "/home/stephbuon/miniconda/lib/python3.8/http/client.py", line 1332, in getresponse
    response.begin()
  File "/home/stephbuon/miniconda/lib/python3.8/http/client.py", line 303, in begin
    version, status, reason = self._read_status()
  File "/home/stephbuon/miniconda/lib/python3.8/http/client.py", line 264, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/home/stephbuon/miniconda/lib/python3.8/socket.py", line 669, in readinto
    return self._so

In [ ]:
#custom tiles: https://stackoverflow.com/questions/62621475/python-folium-custom-tile-setting 
#hover mouse: https://stackoverflow.com/questions/41095716/hover-in-popup-in-folium
#limiter error: https://stackoverflow.com/questions/58439692/convert-physical-addresses-to-geographic-locations-latitude-and-longitud

In [7]:
color_pallet = {
    1830:'#DFFF00', 
    1840:'#FFBF00', 
    1850:'#FF7F50',
    1860:'#DE3163',
    1870:'#9FE2BF',
    1880:'#6495ED'}


In [8]:
#color_pallete = sns.color_palette()
#color_pallete = sns.color_palette('Set2', 8000)
#color_pallete = color_pallete.as_hex()

#print([i for i in color_pallete])

#c = color_pallete[int(row['decade'])] # for seaborn option use this in the expression instead

map1 = folium.Map(
    location=[41.925098, -74.026143], # starting location on map
    tiles='cartodbpositron',
    zoom_start=8,
    #min_zoom=8,
)

for index, row in df.iterrows():
    for k, v in color_pallet.items():
        if k == row['decade']:
            c = v
            folium.CircleMarker([row['latitude'], row['longitude']], popup = row['address'], fill_color=c, fill=True, color=c).add_to(map1)


In [ ]:
# https://stackoverflow.com/questions/65042654/how-to-add-categorical-legend-to-python-folium-map

In [10]:
from legend import add_categorical_legend
map1 = add_categorical_legend(map1, 'Decades', colors = color_pallet.values(), labels = color_pallet.keys())

In [11]:
map1

In [92]:
map1.save('THADS_map_' + d_today + '.html')
